# Pandas의 기초와 고속화 tip

기초 reference : https://doorbw.tistory.com/172

고속화 reference : https://aldente0630.github.io/data-science/2018/08/05/a-beginners-guide-to-optimizing-pandas-code-for-speed.html

## 1. Pandas란?

In [1]:
import pandas as pd
import numpy as np

### Pandas의 자료구조 

Pandas에서는 기본적으로 정의되는 자료구조인 Series와 Data frame을 사용한다.
이 자료구조들은 거대한 데이터를 분석하는 데 있어서 높은 수준의 성능을 보여준다. 

### 1) Series

    - index : 각 저장된 값을 불러올 수 있는 지표 (dictionary 같은 것이라고 생각하면 될까?) 
    - values : 실질적인 값

In [19]:
obj = pd.Series([4, 7, -5, 3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [20]:
obj.values

array([ 4,  7, -5,  3], dtype=int64)

In [26]:
obj.index

RangeIndex(start=0, stop=4, step=1)

1-1. Series의 index는 바꿀 수 있다.

In [22]:
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [28]:
obj2.values

array([ 4,  7, -5,  3], dtype=int64)

In [27]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

1-2. Dictionary 자료형을 Series data로 만들 수 있다. 

    - Dictionary의 key = Series의 Index
    - Dictionary의 value = Series의 Values

In [24]:
sdata = {'Kim':35000, 'Beonwoo':67000, 'Joan':12000, 'Choi':4000}
sdata

{'Kim': 35000, 'Beonwoo': 67000, 'Joan': 12000, 'Choi': 4000}

In [30]:
obj3 = pd.Series(sdata)
obj3

Kim        35000
Beonwoo    67000
Joan       12000
Choi        4000
dtype: int64

In [31]:
obj3.index

Index(['Kim', 'Beonwoo', 'Joan', 'Choi'], dtype='object')

In [32]:
obj3.values

array([35000, 67000, 12000,  4000], dtype=int64)

In [33]:
obj3.name = 'Salary'
obj3.index.name = "Names"
obj3

Names
Kim        35000
Beonwoo    67000
Joan       12000
Choi        4000
Name: Salary, dtype: int64

In [34]:
obj3.index=['A', 'B', 'C', 'D']
obj3

A    35000
B    67000
C    12000
D     4000
Name: Salary, dtype: int64

### 2) Dataframe

Dataframe은 dictionary 또는 numpy의 array로 정의할 수 있다. 

- Dictionary로 정의하는 경우 

In [37]:
data = {'name' : ['Beomwoo', 'Beomwoo', 'Beomwoo', "Kim", "Park"],
       'year' : [2013, 2014, 2015, 2016, 2015],
       'points' : [1.5, 1.7, 3.6, 2.4, 2.9]}
df = pd.DataFrame(data)
df

,name,year,points
0,Beomwoo,2013,1.5
1,Beomwoo,2014,1.7
2,Beomwoo,2015,3.6
3,Kim,2016,2.4
4,Park,2015,2.9


DataFrame의 경우 행과 열을 온전히 가진 자료 구조가 생성된다.

* 행 방향의 index

In [38]:
df.index

RangeIndex(start=0, stop=5, step=1)

* 열 방향의 index

In [39]:
df.values

array([['Beomwoo', 2013, 1.5],
       ['Beomwoo', 2014, 1.7],
       ['Beomwoo', 2015, 3.6],
       ['Kim', 2016, 2.4],
       ['Park', 2015, 2.9]], dtype=object)

* 각 인덱스에 대한 이름을 설정할 수 있다.

In [45]:
df.index.name = "Num"
df.columns.name = "info"
df

info,name,year,points
Num,,,
0,Beomwoo,2013,1.5
1,Beomwoo,2014,1.7
2,Beomwoo,2015,3.6
3,Kim,2016,2.4
4,Park,2015,2.9


* DataFrame을 만들면서 columns와 index를 설정할 수 있다.

In [48]:
df2 = pd.DataFrame(data, columns=['year', 'name', 'points', 'penalty'], index=['one', 'two', 'three', 'four', 'five'])
df2

,year,name,points,penalty
one,2013,Beomwoo,1.5,NaN
two,2014,Beomwoo,1.7,NaN
three,2015,Beomwoo,3.6,NaN
four,2016,Kim,2.4,NaN
five,2015,Park,2.9,NaN


* decribe() 함수를 이용하여 다양한 계산값을 빠르게 확인할 수 있다.

In [49]:
df2.describe()

,year,points
count,5.000000,5.000000
mean,2014.600000,2.420000
std,1.140175,0.864292
min,2013.000000,1.500000
25%,2014.000000,1.700000
50%,2015.000000,2.400000
75%,2015.000000,2.900000
max,2016.000000,3.600000


# 2. Pandas 고속화 실습

In [2]:
df = pd.read_csv('./new_york_hotels.csv')
df.head()

,ean_hotel_id,name,address1,city,state_province,postal_code,latitude,longitude,star_rating,high_rate,low_rate
0,269955,Hilton Garden Inn Albany/SUNY Area,1389 Washington Ave,Albany,NY,12206,42.68751,-73.81643,3.0,154.0272,124.0216
1,113431,Courtyard by Marriott Albany Thruway,1455 Washington Avenue,Albany,NY,12206,42.68971,-73.82021,3.0,179.0100,134.0000
2,108151,Radisson Hotel Albany,205 Wolf Rd,Albany,NY,12205,42.72410,-73.79822,3.0,134.1700,84.1600
3,254756,Hilton Garden Inn Albany Medical Center,62 New Scotland Ave,Albany,NY,12208,42.65157,-73.77638,3.0,308.2807,228.4597
4,198232,CrestHill Suites SUNY University Albany,1415 Washington Avenue,Albany,NY,12206,42.68873,-73.81854,3.0,169.3900,89.3900


In [3]:
df.shape

(1631, 11)

In [15]:
def haversine(lat1, lon1, lat2 ,lon2):
    # calculate distance between two coordinates
    # lat = latitude
    # lon = longitude
    MILES = 3959
    lat1 ,lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    total_miles = MILES * c
    return total_miles

In [16]:
haversine(40.671, -73.985, df.loc[0, 'latitude'], df.loc[0, 'longitude'])

139.60718990468314